In [1]:
import pickle
import pandas as pd

#### Identificator to sequence mapping

In [2]:
id2seq = {}
aa1 = "ACDEFGHIKLMNPQRSTVWY"
from Bio import SeqIO
for record in SeqIO.parse("/home/db/pdb/pdb_seqres_clean.txt", "fasta"):
    nonstd_present = False
    for aa in str(record.seq):
        if aa not in aa1:
            nonstd_present = True
            break
    if not nonstd_present:
        id2seq[record.id] = str(record.seq)

#### Coils output parser

In [3]:
def parse_coils(fn):
    f = open(fn)
    data = f.readlines()
    assignment = []
    seq = []
    for i in range(2, len(data)):
        line = data[i].rstrip()
        results = line.split()
        if float(results[4]) > 0.5:
            assignment.append('1')
        else:
            assignment.append('0')
        seq.append(results[1])
    return(''.join(seq), ''.join(assignment))

#### Get coiled coil assignments for 3 different window variants (14, 21, 28)

In [4]:
df = pickle.load(open('./../../1_Data_Preparation/out/pickle/data_all_74.p', 'rb'))

In [5]:
entries = set(df.index.tolist())

In [6]:
variants = ['14', '21', '28']
pcoils_results = {}
for entry in entries:
    if entry in id2seq.keys():
        results = []
        for variant in variants:
            result = '0'*len(id2seq[entry])
            try:
                result = parse_coils('results/%s.coils_n%s' % (entry, variant))
                assert result[0] == id2seq[entry]
            except FileNotFoundError:
                pass
            results.append(result[1])
            if '1' in result[1]:
                results.append(1)
            else:
                results.append(0)
        pcoils_results[entry] = results

In [7]:
len(entries), len(pcoils_results)

(28939, 28939)

In [8]:
df = pd.DataFrame.from_dict(pcoils_results, orient='index')
df.columns = ['coils_14_assignment', 'coils_14_cc',
              'coils_21_assignment', 'coils_21_cc',
             'coils_28_assignment', 'coils_28_cc']

In [9]:
df.to_csv('./../../1_Data_Preparation/out/csv/coils_all_74.csv')
df.to_pickle(open('./../../1_Data_Preparation/out/pickle/coils_all_74.p', 'wb'))